In [460]:
import pandas as pd
import numpy as np
import hijri_converter


In [461]:
from hijri_converter import convert
def convert_date(date_string):
    return pd.to_datetime(date_string, format='%d/%m/%Y')

df = pd.read_excel("RequiredData.xlsx", sheet_name="Reservation List", converters={'Reservation Check-out Date ':convert_date  , 'Cancellation Date':convert_date , 'Reservation Check-in Date':convert_date ,'Reservation Creation Date (MM-DD-YYYY)':convert_date})

df['Hijri Check-out Date'] = df['Reservation Check-out Date '].apply(lambda x: convert.Gregorian(x.year, x.month, x.day).to_hijri())
df['Hijri Check-in Date'] = df['Reservation Check-in Date'].apply(lambda x: convert.Gregorian(x.year, x.month, x.day).to_hijri())


df.sort_values('Reservation Creation Date (MM-DD-YYYY)', inplace=True)
df.head(10)

,Reservation ID,Reservation Creation Date (MM-DD-YYYY),Reservation Check-in Date,Reservation Check-out Date,Reservation Total Price,#Rooms,Cancellation Date,"Type\nallotment (1), sales allotment (2), direct hotel (3)or direct agent(4)",Hijri Check-out Date,Hijri Check-in Date
0,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11
1,2,2023-01-01,2023-05-04,2023-05-09,5000,5,NaT,1,1444-10-19,1444-10-14
2,3,2023-01-01,2023-08-09,2023-08-10,200,5,2023-01-03,1,1445-01-23,1445-01-22
3,4,2023-01-02,2023-09-07,2023-09-16,7200,4,2023-01-03,1,1445-03-01,1445-02-22
4,5,2023-01-02,2023-01-09,2023-01-12,81400,20,NaT,2,1444-06-19,1444-06-16
5,6,2023-01-03,2023-04-21,2023-04-27,74243,55,NaT,1,1444-10-07,1444-10-01


In [462]:
from hijri_converter import Hijri, Gregorian
Hijri(1433, 9, 1)

Hijri(1433, 9, 1)

In [463]:
value_1 = df.loc[0,'Hijri Check-out Date']
value_2 = df.loc[5,'Hijri Check-out Date']


if value_1 > value_2:
    print("The first row is bigger.")
elif value_1 < value_2:
    print("The first row is smaller.")
else:
    print("The values are equal.")

The first row is bigger.


In [464]:
# Create a new DataFrame to store the filtered reservations
filtered_df = pd.DataFrame(columns=df.columns)


In [465]:
target_start_day_hijri = Hijri(1444,9,1) # Hijri date in the format 'yyyy-mm-dd'
target_end_day_hijri  = Hijri(1445,1,1) # Hijri date in the format 'yyyy-mm-dd'
is_different_year = (target_start_day_hijri.year!=target_end_day_hijri.year)

In [466]:
def hijri_to_greg(hijri_date):
    dt_greg = hijri_date.to_gregorian().ctime()
    dt_greg = pd.to_datetime(dt_greg)
    return dt_greg

def increment_hijri(hijri_date, dateOffset=1):
    dt_greg = hijri_to_greg(hijri_date)
    dt_greg += pd.DateOffset(days=dateOffset)
    incremented_hijri_date = Gregorian(dt_greg.year, dt_greg.month, dt_greg.day).to_hijri()
    return incremented_hijri_date

In [467]:
# Iterate over each row in the original DataFrame
for index, row in df.iterrows():
    # Extract the check-in and check-out  Hijri dates 
    check_in_date_hijri = row['Hijri Check-in Date']
    check_out_date_hijri = row['Hijri Check-out Date']
    target_start_day_hijri_temp = Hijri(check_in_date_hijri.year, target_start_day_hijri.month, target_start_day_hijri.day) 
    target_end_day_hijri_temp = Hijri(check_out_date_hijri.year + is_different_year, target_end_day_hijri.month, target_end_day_hijri.day)



# Create a list to store the rows for each day within the range
    rows = []

    # Iterate over each day within the range
    current_day = check_in_date_hijri
    while current_day <= check_out_date_hijri:
        # Check if the current day falls within the check-in and check-out dates
        if target_start_day_hijri_temp <= current_day <= target_end_day_hijri_temp:
            # Append the row with the current day to the list
            rows.append(list(row)+[current_day])

        current_day = increment_hijri(current_day)

    if (rows == []):
        continue

    # Create a DataFrame for the rows for this reservation
    reservation_df = pd.DataFrame(rows, columns=list(df.columns)+['Target Date'])

    # Concatenate the reservation DataFrame with the filtered DataFrame
    filtered_df = pd.concat([filtered_df, reservation_df], ignore_index=True)



In [468]:
filtered_df

,Reservation ID,Reservation Creation Date (MM-DD-YYYY),Reservation Check-in Date,Reservation Check-out Date,Reservation Total Price,#Rooms,Cancellation Date,"Type\nallotment (1), sales allotment (2), direct hotel (3)or direct agent(4)",Hijri Check-out Date,Hijri Check-in Date,Target Date
0,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-11
1,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-12
2,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-13
3,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-14
4,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-15
5,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-16
6,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-17
7,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-18
8,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-19
9,1,2023-01-01,2023-05-01,2023-05-12,24000,10,NaT,1,1444-10-22,1444-10-11,1444-10-20


In [469]:
numeric_columns = [col for col in filtered_df.columns if 'date' not in col.lower()]
filtered_df[numeric_columns] = filtered_df[numeric_columns].apply(pd.to_numeric, errors='coerce').astype('Int64')
filtered_df.dtypes

Reservation ID                                                                            Int64
Reservation Creation Date (MM-DD-YYYY)                                           datetime64[ns]
Reservation Check-in Date                                                        datetime64[ns]
Reservation Check-out Date                                                       datetime64[ns]
Reservation Total Price                                                                   Int64
#Rooms                                                                                    Int64
Cancellation Date                                                                datetime64[ns]
Type\nallotment (1),  sales allotment (2), direct hotel (3)or direct agent(4)             Int64
Hijri Check-out Date                                                                     object
Hijri Check-in Date                                                                      object
Target Date                             

In [470]:
ucdts = filtered_df['Reservation Creation Date (MM-DD-YYYY)'].unique()
ucdts

<DatetimeArray>
['2023-01-01 00:00:00', '2023-01-03 00:00:00']
Length: 2, dtype: datetime64[ns]

In [471]:
def get_cr_dates_per_year(df):
    """
    Retrieve a list of dataframe arrays containing reservation creation dates grouped by year.

    Parameters:
        df (pandas.DataFrame): The initial dataframe.

    Returns:
        list: A list of dataframe arrays, where each array contains reservation creation dates for a specific year.

    Example:
        >>> df = pd.DataFrame({'Reservation Creation Date (MM-DD-YYYY)': ['01-01-2022', '01-02-2022', '01-01-2023', '01-02-2023']})
        >>> cr_dates_per_year = get_cr_dates_per_year(df)
        >>> print(cr_dates_per_year)
        [['01-01-2022', '01-02-2022'], ['01-01-2023', '01-02-2023']]
    """
    df = df.copy()
    # Group the dataframe array by year
    grouped = df.groupby(df['Reservation Creation Date (MM-DD-YYYY)'].dt.year)

    # Create separate dataframe arrays for each year
    cr_dates_per_year = [group['Reservation Creation Date (MM-DD-YYYY)'].tolist() for _, group in grouped]

    return cr_dates_per_year

In [472]:
def create_reservation_dataframe(year):
    """
    Create a dataframe with a column 'Reservation Creation Date' containing all the days in a specified year.

    Parameters:
        year (int): The year for which to create the dataframe.

    Returns:
        pandas.DataFrame: A dataframe with a single column 'Reservation Creation Date' containing all the days in the specified year.

    Example:
        >>> df = create_reservation_dataframe(2023)
        >>> print(df)
           Reservation Creation Date
        0                2023-01-01
        1                2023-01-02
        2                2023-01-03
        ...              ...
        364              2023-12-31
    """
    start_date = pd.to_datetime(f'{year}-01-01')
    end_date = pd.to_datetime(f'{year}-12-31')

    dates = pd.date_range(start=start_date, end=end_date, freq='D')

    df = pd.DataFrame({'Reservation Creation Date': dates})

    return df


def greg_to_hijri(greg_date):
    hijri_date = Gregorian(greg_date.year, greg_date.month, greg_date.day)
    hijri_date = hijri_date.to_hijri()
    return hijri_date

In [473]:
cr_dates_per_year = get_cr_dates_per_year(filtered_df)
target_dates_history_years = []
target_dates_history_dfs = []
for annually_creation_dates_greg in cr_dates_per_year:
    target_start_day_hijri_temp = Hijri(greg_to_hijri(annually_creation_dates_greg[0]).year, 
                                        target_start_day_hijri.month, 
                                        target_start_day_hijri.day)
    target_end_day_hijri_temp = Hijri(greg_to_hijri(annually_creation_dates_greg[0]).year + is_different_year, 
                                        target_end_day_hijri.month, 
                                        target_end_day_hijri.day)
    
    target_dates_range = []
    current_day = target_start_day_hijri_temp
    while current_day <= target_end_day_hijri_temp:
        target_dates_range.append(current_day)
        current_day = increment_hijri(current_day)

    # Creating empty df
    annually_target_dates_history_df = create_reservation_dataframe(annually_creation_dates_greg[0].year)
    # Creates columns where the column name is a Hijri() object (displayed as Y-m-d by __str()__ method in Hijri class)
    for column in target_dates_range:
        annually_target_dates_history_df[column] = np.nan
    
    for creation_date_greg in annually_creation_dates_greg:
        target_creation_dates_mask = (annually_target_dates_history_df['Reservation Creation Date'] == creation_date_greg)
        filtered_creation_dates_mask = (filtered_df['Reservation Creation Date (MM-DD-YYYY)'] == creation_date_greg)
        filtered_target_dates = [dt for dt in list(filtered_df[filtered_creation_dates_mask]['Target Date'])]
        filtered_rooms = list(filtered_df[filtered_creation_dates_mask]['#Rooms'])
        
        target_date_to_rooms = {}
        for i, target_date in enumerate(filtered_target_dates):
            if target_date not in target_date_to_rooms.keys():
                target_date_to_rooms[target_date] = 0
            target_date_to_rooms[target_date] += filtered_rooms[i]    

        annually_target_dates_history_df.loc[target_creation_dates_mask, # to select row in ann_td_h_df where creation date is the current one in the loop
                                                list(target_date_to_rooms.keys()) # to update the column names of said row
                                            ] \
                                            = list(target_date_to_rooms.values()) # updates it to #Rooms
         

    target_dates_history_years.append(annually_creation_dates_greg[0].year)
    target_dates_history_dfs.append(annually_target_dates_history_df)

target_dates_history_years

C:\Users\ashra\AppData\Local\Temp\ipykernel_46140\1743774333.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  annually_target_dates_history_df[column] = np.nan
C:\Users\ashra\AppData\Local\Temp\ipykernel_46140\1743774333.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  annually_target_dates_history_df[column] = np.nan
C:\Users\ashra\AppData\Local\Temp\ipykernel_46140\1743774333.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfo

[2023]

In [474]:
target_dates_accum_history_dfs = target_dates_history_dfs.copy()
for tdacc_df in target_dates_accum_history_dfs:
    # cumulatively add each row to the row below it
    tdacc_df.fillna(0, inplace=True)
    tdacc_df[tdacc_df.columns[1:]] = tdacc_df[tdacc_df.columns[1:]].cumsum()

    # when reaching the row where "Reservation Creation Date" is equal to the Column date, 
    # fill that column's remaining rows (after that row) with 0s 
    for i, column_hijri in enumerate(tdacc_df.columns):
        if not isinstance(column_hijri, Hijri):
            continue 
        matching_dates_row_idx = tdacc_df.loc[tdacc_df['Reservation Creation Date']==hijri_to_greg(column_hijri)].index[0]
        if matching_dates_row_idx == len(tdacc_df)-1:
            continue
        tdacc_df.iloc[matching_dates_row_idx+1:, i] = np.nan

In [475]:
def display_full_df(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(df)

In [476]:
target_dates_accum_history_dfs[0]

,Reservation Creation Date,1444-09-01,1444-09-02,1444-09-03,1444-09-04,1444-09-05,1444-09-06,1444-09-07,1444-09-08,1444-09-09,...,1444-12-22,1444-12-23,1444-12-24,1444-12-25,1444-12-26,1444-12-27,1444-12-28,1444-12-29,1444-12-30,1445-01-01
0,2023-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2023-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361,2023-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,2023-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
363,2023-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [477]:
weekly_target_dates_accum_dfs = []
for tdacc_df in target_dates_accum_history_dfs:
    new_columns = ['Reservation Creation Date']
    new_agg_values = [tdacc_df.iloc[:, 0].to_list()]
    df_columns = tdacc_df.columns
    for start_date_idx in range(1, len(df_columns), 7):
        # get column name of this week's range
        end_date_idx = min(start_date_idx+6, len(df_columns)-1)
        column_name = f'W{(start_date_idx//7)%4 + 1} - ' + df_columns[start_date_idx].month_name()
        if df_columns[start_date_idx].month != df_columns[end_date_idx].month:
            column_name += '/' + df_columns[end_date_idx].month_name()
        new_columns.append(column_name)

        # get sum of #Rooms of this week's range as a single list
        new_agg_values.append(tdacc_df.iloc[:, start_date_idx:start_date_idx+7].sum(axis=1, skipna=True).to_list())

    weekly_tdacc_df = pd.DataFrame(np.array(new_agg_values).T, columns=new_columns)

    # convert all trailing zero values (per column) to np.NaN values
    for column in weekly_tdacc_df.columns:
        if 'date' in column.lower():
            continue
        mask = weekly_tdacc_df[column] != 0
        last_nonzero_index = mask[::-1].idxmax()
        if last_nonzero_index < len(weekly_tdacc_df)-1:
            weekly_tdacc_df.loc[last_nonzero_index + 1:, column] = np.nan

    weekly_target_dates_accum_dfs.append(weekly_tdacc_df)


In [478]:
weekly_target_dates_accum_dfs[0]

,Reservation Creation Date,W1 - Ramadhan,W2 - Ramadhan,W3 - Ramadhan,W4 - Ramadhan,W1 - Ramadhan/Shawwal,W2 - Shawwal,W3 - Shawwal,W4 - Shawwal,W1 - Shawwal/Dhu al-Qi’dah,W2 - Dhu al-Qi’dah,W3 - Dhu al-Qi’dah,W4 - Dhu al-Qi’dah,W1 - Dhu al-Qi’dah/Dhu al-Hijjah,W2 - Dhu al-Hijjah,W3 - Dhu al-Hijjah,W4 - Dhu al-Hijjah,W1 - Dhu al-Hijjah/Muharram
0,2023-01-01,0.0,0.0,0.0,0.0,0.0,30.0,100.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023-01-02,0.0,0.0,0.0,0.0,0.0,30.0,100.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023-01-03,0.0,0.0,0.0,0.0,330.0,85.0,100.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-01-04,0.0,0.0,0.0,0.0,330.0,85.0,100.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-01-05,0.0,0.0,0.0,0.0,330.0,85.0,100.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2023-12-27,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
361,2023-12-28,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
362,2023-12-29,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
363,2023-12-30,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
